## Imports

In [10]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time
import googlemaps

## Read-in

In [2]:
df = pd.read_csv('Tauber - FOR MAP.csv')

In [5]:
df = df.rename(columns={'Unnamed: 12':'Deed Link'})

In [11]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

%store -r map_box_api_key

In [12]:
df['Full_Address'] = df['Address'] + ' Chicago, IL'

In [20]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

In [21]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## HTML Popup Formatter

In [13]:
pd.set_option('display.max_columns',None)

In [14]:
df.head(1)

,Owner,Address,Number of Units,Purchase Amount,Purchase Date,Mortgage Amount,Loan Originator,Current Mortgage Holder,Still Own?,In Foreclosure?,Other Lawsuits?,Board of Review 2024 value,Deed Link,Full_Address
0,ETC5 LLC,9130 S. Throop St.,4,"$680,000",May 2024,"$544,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$340,000",https://crs.cookcountyclerkil.gov/Document/Det...,"9130 S. Throop St. Chicago, IL"


In [17]:
def popup_html(row):
    owner             = row['Owner']
    address           = row['Address']
    num_units         = row['Number of Units']
    purchase_amount   = row['Purchase Amount']
    purchase_date     = row['Purchase Date']
    mortgage_amount   = row['Mortgage Amount']
    loan_originator   = row['Loan Originator']
    current_m_holder  = row['Current Mortgage Holder']
    bor_2024_value    = row['Board of Review 2024 value']
    
    # Format currency fields with commas and two decimals
    fmt = lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else x

    html = f"""<!DOCTYPE html>
<html>
  <body>
    <strong>Owner:</strong> {owner}<br>
    <strong>Address:</strong> {address}<br>
    <strong>Number of Units:</strong> {num_units}<br>
    <strong>Purchase Amount:</strong> ${fmt(purchase_amount)}<br>
    <strong>Purchase Date:</strong> {purchase_date}<br>
    <strong>Mortgage Amount:</strong> ${fmt(mortgage_amount)}<br>
    <strong>Loan Originator:</strong> {loan_originator}<br>
    <strong>Current Mortgage Holder:</strong> {current_m_holder}<br>
    <strong>Board of Review 2024 Value:</strong> ${fmt(bor_2024_value)}<br>
  </body>
</html>
"""
    return html


In [122]:
%store -r map_box_api_key

In [22]:
df

,Owner,Address,Number of Units,Purchase Amount,Purchase Date,Mortgage Amount,Loan Originator,Current Mortgage Holder,Still Own?,In Foreclosure?,Other Lawsuits?,Board of Review 2024 value,Deed Link,Full_Address,geocoded,lat,lon
0,ETC5 LLC,9130 S. Throop St.,4,"$680,000",May 2024,"$544,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$340,000",https://crs.cookcountyclerkil.gov/Document/Det...,"9130 S. Throop St. Chicago, IL","(41.7276646, -87.65588939999999)",41.727665,-87.655889
1,ETS2 LLC,8017 S. Hermitage Ave.,2,"$495,000",March 2024,"$400,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$180,000",https://crs.cookcountyclerkil.gov/Document/Det...,"8017 S. Hermitage Ave. Chicago, IL","(41.7479542, -87.6667587)",41.747954,-87.666759
2,"CCP1, LLC",5226 S. Paulina St.,4,"$485,000",Dec. 2023,"$364,000",Ark Mortgage,"Wilmington Savings Fund Scoiety, as owner trustee",Y,Y,NaN,"$460,000",https://crs.cookcountyclerkil.gov/Document/Det...,"5226 S. Paulina St. Chicago, IL","(41.7986639, -87.66739489999999)",41.798664,-87.667395
3,ETS2 LLC,8052 S. Langley Ave.,2,"$495,000",Feb. 2024,"$396,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$167,000",https://crs.cookcountyclerkil.gov/Document/Det...,"8052 S. Langley Ave. Chicago, IL","(41.74788360000001, -87.60788389999999)",41.747884,-87.607884
4,ETS LLC,8047 S. Vernon Ave.,3,"$495,000",Feb. 2024,"$496,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$173,000",https://crs.cookcountyclerkil.gov/Document/Det...,"8047 S. Vernon Ave. Chicago, IL","(41.7479549, -87.61334660000001)",41.747955,-87.613347
5,ETC1 LLC,5621 S. Green St.,8,"$680,000",May 2024,"$544,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$235,000",https://crs.cookcountyclerkil.gov/Document/Det...,"5621 S. Green St. Chicago, IL","(41.7918269, -87.6460505)",41.791827,-87.646051
6,ETS1 LLC,7944 S. Eberhart Ave.,2,"$495,000",Feb. 2024,"$371,000",Ark Mortgage,"Wilmington Savings Fund Scoiety, as owner trustee",Y,Y,NaN,"$165,000",https://crs.cookcountyclerkil.gov/Document/Det...,"7944 S. Eberhart Ave. Chicago, IL","(41.7498099, -87.6127876)",41.749810,-87.612788
7,ETS2 LLC,7938 S. Langley Ave.,2,"$495,000",Feb. 2024,"$396,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$154,000",https://crs.cookcountyclerkil.gov/Document/Det...,"7938 S. Langley Ave. Chicago, IL","(41.7500783, -87.6079344)",41.750078,-87.607934
8,ETS1 LLC,7937 S. Eberhart Ave.,2,"$495,000",Feb. 2024,"$371,000",Ark Mortgage,"Wilmington Savings Fund Scoiety, as owner trustee",Y,Y,NaN,"$148,000",https://crs.cookcountyclerkil.gov/Document/Det...,"7937 S. Eberhart Ave. Chicago, IL","(41.7500776, -87.6121723)",41.750078,-87.612172


In [26]:
import folium
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement

# ——— Base map ———
m = folium.Map(
    location=df[['lat','lon']].mean().to_list(),
    zoom_start=12,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Create a MarkerCluster and add it to the map ———
marker_cluster = MarkerCluster(name="Properties").add_to(m)

# ——— Add your markers into that cluster group ———
for _, row in df.iterrows():
    lat, lon = row['lat'], row['lon']
    popup = folium.Popup(popup_html(row), max_width=400)
    # You can still use CircleMarker inside a cluster:
    folium.CircleMarker(
        location=(lat, lon),
        radius=7,
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        color=None,
        popup=popup
    ).add_to(marker_cluster)

# ——— (Optional) Add LayerControl so you can toggle clusters on/off ———
folium.LayerControl().add_to(m)

m  # display


In [124]:
m.save('index.html')

## Data snagger

In [125]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [126]:
df_top_ten.head(1)

,index,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,116,PAST SALE,2025-07-30,Condo/Co-op,2770 S Ocean Blvd Unit 201n,Palm Beach,FL,33480.0,4000000.0,4.0,5.5,2770,4370.0,NaN,1991.0,NaN,915.0,5750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-10936399,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Carole Koeppel,"Sotheby's Intl. Realty, Inc.",None,None,Dana Koch,The Corcoran Group,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [127]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_08042025 


# CREATE TEMPLATE 

In [128]:
muni_set = set(df_top_ten['CITY'])

In [129]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [130]:
df_top_ten.reset_index(inplace=True,drop=True)

In [131]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [132]:
top_sale

'2770 at 2770 S Ocean Blvd Unit 201n in Palm Beach'

In [133]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [134]:
df_top_ten.columns

Index(['index', 'SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [135]:
df_top_ten

,index,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,116,PAST SALE,2025-07-30,Condo/Co-op,2770 S Ocean Blvd Unit 201n,Palm Beach,FL,33480.0,4000000.0,4.0,5.5,2770,4370.0,NaN,1991.0,NaN,915.0,5750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-10936399,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Carole Koeppel,"Sotheby's Intl. Realty, Inc.",None,None,Dana Koch,The Corcoran Group,None,None,1,orange,2770 at 2770 S Ocean Blvd Unit 201n in Palm Beach
1,44,PAST SALE,2025-07-30,Condo/Co-op,2600 S Ocean Blvd Unit 9C & 9D,Boca Raton,FL,33432.0,3100000.0,4.0,4.5,OCEANS EDGE,4202.0,NaN,1974.0,NaN,738.0,3926.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2600-S-Oc...,MARMLS,A11778186,N,Y,26.325422,-80.074701,https://www.redfin.com/FL/Boca-Raton/2600-S-Oc...,Devin Kay,Douglas Elliman,None,None,None,None,None,None,2,blue,OCEANS EDGE at 2600 S Ocean Blvd Unit 9C & 9D ...
2,18,PAST SALE,2025-07-31,Condo/Co-op,701 Captains Way #701,Jupiter,FL,33477.0,2100000.0,3.0,2.0,Captains Way AT Admirals Cove Condo,2033.0,NaN,1988.0,NaN,1033.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/701-Captains...,Beaches MLS,RX-11096443,N,Y,26.907510,-80.085826,https://www.redfin.com/FL/Jupiter/701-Captains...,Lisa Fishman,"NV Realty Group, LLC",None,None,Jessica Kaufman,Beachfront Realty Inc,None,None,3,blue,Captains Way AT Admirals Cove Condo at 701 Cap...
3,8,PAST SALE,2025-07-31,Condo/Co-op,19630 Sawgrass Cir #2901,Boca Raton,FL,33434.0,1300000.0,3.0,3.0,Peppertree,2080.0,NaN,1979.0,NaN,625.0,1460.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/19630-Saw...,Beaches MLS,RX-11098622,N,Y,26.390498,-80.155550,https://www.redfin.com/FL/Boca-Raton/19630-Saw...,Judith Romanow,Boca West Realty LLC,Ina Tuchman,Boca West Realty LLC,Ina Tuchman,Boca West Realty LLC,None,None,4,blue,Peppertree at 19630 Sawgrass Cir #2901 in Boca...
4,91,PAST SALE,2025-07-29,Condo/Co-op,3115 S Ocean Blvd #1004,Highland Beach,FL,33487.0,1250000.0,3.0,2.5,Ocean Terrace North Condo,1852.0,NaN,1981.0,NaN,675.0,2500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3115-...,Beaches MLS,RX-11061970,N,Y,26.418745,-80.063926,https://www.redfin.com/FL/Highland-Beach/3115-...,Colleen Newland,Douglas Elliman,None,None,Scott Correale,One Sotheby's International Realty,None,None,5,blue,Ocean Terrace North Condo at 3115 S Ocean Blvd...
5,49,PAST SALE,2025-07-31,Condo/Co-op,155 Tresana Blvd #107,Jupiter,FL,33478.0,1250000.0,3.0,2.5,Jupiter Country Club Condo II,3042.0,NaN,2017.0,NaN,411.0,1295.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/155-Tresana-...,Beaches MLS,RX-11074044,N,Y,26.928057,-80.155226,https://www.redfin.com/FL/Jupiter/155-Tresana-...,Gray Welton,"NV Realty Group, LLC",None,None,Jennifer Friedman,Fine Living Realty of Palm Beaches,None,None,6,blue,Jupiter Country Club Condo II at 155 Tresana B...
6,21,PAST SALE,2025-08-01,Condo/Co-op,4740 S Ocean Blvd #1403,Highland Beach,FL,33487.0,812000.0,2.0,2.0,Braemar Isle Condo,2050.0,NaN,1982.0,NaN,396.0,2597.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11097738,N,Y,26.393265,-80.069030,https://www.redfin.com/FL/Highland-Beach/4740-...,Ryan Coleman,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Mary Ann Phillips,RE/MAX Select Group,None,None,7,blue,Braemar Isle Condo at 4740 S Ocean Blvd #1403 ...
7,15,PAST SALE,2025-07-30,Condo/Co-op,11282 Glen Oaks Ct,North Palm Beach,FL,33408.0,755000.0,2.0,2.5,North Palm Beach,2300.0,NaN,1983.0,NaN,328.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/112...

In [136]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name.strip()}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 2770 closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $630,000 to $4,000,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, West Palm Beach, Highland Beach, Boca Raton, Palm Beach, North Palm Beach, Jupiter, Boynton Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 94 condo sales totaling $37,140,300 million from July 28 to Aug. 3. The previous week, brokers closed 57 condo sales totaling $23,474,515.

Last week’s units sold for an aver

In [137]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [138]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [139]:
df_top_ten.columns

Index(['index', 'SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [70]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-30,Condo/Co-op,2770 S Ocean Blvd Unit 201n,Palm Beach,FL,33480.0,4000000.0,4.0,5.5,2770,4370.0,NaN,1991.0,NaN,915.0,5750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-10936399,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Carole Koeppel,"Sotheby's Intl. Realty, Inc.",None,None,Dana Koch,The Corcoran Group,None,None,1,orange,2770 at 2770 S Ocean Blvd Unit 201n in Palm Beach
1,PAST SALE,2025-07-30,Condo/Co-op,2600 S Ocean Blvd Unit 9C & 9D,Boca Raton,FL,33432.0,3100000.0,4.0,4.5,OCEANS EDGE,4202.0,NaN,1974.0,NaN,738.0,3926.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2600-S-Oc...,MARMLS,A11778186,N,Y,26.325422,-80.074701,https://www.redfin.com/FL/Boca-Raton/2600-S-Oc...,Devin Kay,Douglas Elliman,None,None,None,None,None,None,2,blue,OCEANS EDGE at 2600 S Ocean Blvd Unit 9C & 9D ...
2,PAST SALE,2025-07-31,Condo/Co-op,701 Captains Way #701,Jupiter,FL,33477.0,2100000.0,3.0,2.0,Captains Way AT Admirals Cove Condo,2033.0,NaN,1988.0,NaN,1033.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/701-Captains...,Beaches MLS,RX-11096443,N,Y,26.907510,-80.085826,https://www.redfin.com/FL/Jupiter/701-Captains...,Lisa Fishman,"NV Realty Group, LLC",None,None,Jessica Kaufman,Beachfront Realty Inc,None,None,3,blue,Captains Way AT Admirals Cove Condo at 701 Cap...
3,PAST SALE,2025-07-31,Condo/Co-op,19630 Sawgrass Cir #2901,Boca Raton,FL,33434.0,1300000.0,3.0,3.0,Peppertree,2080.0,NaN,1979.0,NaN,625.0,1460.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/19630-Saw...,Beaches MLS,RX-11098622,N,Y,26.390498,-80.155550,https://www.redfin.com/FL/Boca-Raton/19630-Saw...,Judith Romanow,Boca West Realty LLC,Ina Tuchman,Boca West Realty LLC,Ina Tuchman,Boca West Realty LLC,None,None,4,blue,Peppertree at 19630 Sawgrass Cir #2901 in Boca...
4,PAST SALE,2025-07-29,Condo/Co-op,3115 S Ocean Blvd #1004,Highland Beach,FL,33487.0,1250000.0,3.0,2.5,Ocean Terrace North Condo,1852.0,NaN,1981.0,NaN,675.0,2500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3115-...,Beaches MLS,RX-11061970,N,Y,26.418745,-80.063926,https://www.redfin.com/FL/Highland-Beach/3115-...,Colleen Newland,Douglas Elliman,None,None,Scott Correale,One Sotheby's International Realty,None,None,5,blue,Ocean Terrace North Condo at 3115 S Ocean Blvd...
5,PAST SALE,2025-07-31,Condo/Co-op,155 Tresana Blvd #107,Jupiter,FL,33478.0,1250000.0,3.0,2.5,Jupiter Country Club Condo II,3042.0,NaN,2017.0,NaN,411.0,1295.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/155-Tresana-...,Beaches MLS,RX-11074044,N,Y,26.928057,-80.155226,https://www.redfin.com/FL/Jupiter/155-Tresana-...,Gray Welton,"NV Realty Group, LLC",None,None,Jennifer Friedman,Fine Living Realty of Palm Beaches,None,None,6,blue,Jupiter Country Club Condo II at 155 Tresana B...
6,PAST SALE,2025-08-01,Condo/Co-op,4740 S Ocean Blvd #1403,Highland Beach,FL,33487.0,812000.0,2.0,2.0,Braemar Isle Condo,2050.0,NaN,1982.0,NaN,396.0,2597.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11097738,N,Y,26.393265,-80.069030,https://www.redfin.com/FL/Highland-Beach/4740-...,Ryan Coleman,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Mary Ann Phillips,RE/MAX Select Group,None,None,7,blue,Braemar Isle Condo at 4740 S Ocean Blvd #1403 ...
7,PAST SALE,2025-07-30,Condo/Co-op,11282 Glen Oaks Ct,North Palm Beach,FL,33408.0,755000.0,2.0,2.5,North Palm Beach,2300.0,NaN,1983.0,NaN,328.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/112...,PBBOR,24-1802,N,Y,26.841592,-

In [140]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: 2770 closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $630,000 to $4,000,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, West Palm Beach, Highland Beach, Boca Raton, Palm Beach, North Palm Beach, Jupiter, Boynton Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 94 condo sales totaling $37,140,300 million from July 28 to Aug. 3. The previous week, brokers closed 57 condo sales totaling $23,474,515.

Last week’s units sold for an aver

In [141]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Palm-Beach/2770-S-Ocean-Blvd-33480/unit-201N/home/42437125


In [142]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Boca-Raton/2600-S-Ocean-Blvd-33432/unit-9c-9d/home/193377748


In [143]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Boynton-Beach/3579-S-Quail-Ridge-Dr-33436/unit-Bobwhite-A/home/189530109


In [144]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/West-Palm-Beach/701-S-Olive-Ave-33401/unit-202/home/42661357


In [145]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Jupiter/701-Captains-Way-33477/unit-701/home/195982501


In [146]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/North-Palm-Beach/11282-Glen-Oaks-Ct-33408/home/196028490


In [148]:
df_top_ten

,index,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,116,PAST SALE,2025-07-30,Condo/Co-op,2770 S Ocean Blvd Unit 201n,Palm Beach,FL,33480.0,4000000.0,4.0,5.5,2770,4370.0,NaN,1991.0,NaN,915.0,5750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-10936399,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Carole Koeppel,"Sotheby's Intl. Realty, Inc.",None,None,Dana Koch,The Corcoran Group,None,None,1,orange,2770 at 2770 S Ocean Blvd Unit 201n in Palm Beach
1,44,PAST SALE,2025-07-30,Condo/Co-op,2600 S Ocean Blvd Unit 9C & 9D,Boca Raton,FL,33432.0,3100000.0,4.0,4.5,OCEANS EDGE,4202.0,NaN,1974.0,NaN,738.0,3926.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2600-S-Oc...,MARMLS,A11778186,N,Y,26.325422,-80.074701,https://www.redfin.com/FL/Boca-Raton/2600-S-Oc...,Devin Kay,Douglas Elliman,None,None,None,None,None,None,2,blue,OCEANS EDGE at 2600 S Ocean Blvd Unit 9C & 9D ...
2,18,PAST SALE,2025-07-31,Condo/Co-op,701 Captains Way #701,Jupiter,FL,33477.0,2100000.0,3.0,2.0,Captains Way AT Admirals Cove Condo,2033.0,NaN,1988.0,NaN,1033.0,2357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/701-Captains...,Beaches MLS,RX-11096443,N,Y,26.907510,-80.085826,https://www.redfin.com/FL/Jupiter/701-Captains...,Lisa Fishman,"NV Realty Group, LLC",None,None,Jessica Kaufman,Beachfront Realty Inc,None,None,3,blue,Captains Way AT Admirals Cove Condo at 701 Cap...
3,8,PAST SALE,2025-07-31,Condo/Co-op,19630 Sawgrass Cir #2901,Boca Raton,FL,33434.0,1300000.0,3.0,3.0,Peppertree,2080.0,NaN,1979.0,NaN,625.0,1460.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/19630-Saw...,Beaches MLS,RX-11098622,N,Y,26.390498,-80.155550,https://www.redfin.com/FL/Boca-Raton/19630-Saw...,Judith Romanow,Boca West Realty LLC,Ina Tuchman,Boca West Realty LLC,Ina Tuchman,Boca West Realty LLC,None,None,4,blue,Peppertree at 19630 Sawgrass Cir #2901 in Boca...
4,91,PAST SALE,2025-07-29,Condo/Co-op,3115 S Ocean Blvd #1004,Highland Beach,FL,33487.0,1250000.0,3.0,2.5,Ocean Terrace North Condo,1852.0,NaN,1981.0,NaN,675.0,2500.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3115-...,Beaches MLS,RX-11061970,N,Y,26.418745,-80.063926,https://www.redfin.com/FL/Highland-Beach/3115-...,Colleen Newland,Douglas Elliman,None,None,Scott Correale,One Sotheby's International Realty,None,None,5,blue,Ocean Terrace North Condo at 3115 S Ocean Blvd...
5,49,PAST SALE,2025-07-31,Condo/Co-op,155 Tresana Blvd #107,Jupiter,FL,33478.0,1250000.0,3.0,2.5,Jupiter Country Club Condo II,3042.0,NaN,2017.0,NaN,411.0,1295.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/155-Tresana-...,Beaches MLS,RX-11074044,N,Y,26.928057,-80.155226,https://www.redfin.com/FL/Jupiter/155-Tresana-...,Gray Welton,"NV Realty Group, LLC",None,None,Jennifer Friedman,Fine Living Realty of Palm Beaches,None,None,6,blue,Jupiter Country Club Condo II at 155 Tresana B...
6,21,PAST SALE,2025-08-01,Condo/Co-op,4740 S Ocean Blvd #1403,Highland Beach,FL,33487.0,812000.0,2.0,2.0,Braemar Isle Condo,2050.0,NaN,1982.0,NaN,396.0,2597.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/4740-...,Beaches MLS,RX-11097738,N,Y,26.393265,-80.069030,https://www.redfin.com/FL/Highland-Beach/4740-...,Ryan Coleman,Compass Florida LLC,Scott Gerow,Compass Florida LLC,Mary Ann Phillips,RE/MAX Select Group,None,None,7,blue,Braemar Isle Condo at 4740 S Ocean Blvd #1403 ...
7,15,PAST SALE,2025-07-30,Condo/Co-op,11282 Glen Oaks Ct,North Palm Beach,FL,33408.0,755000.0,2.0,2.5,North Palm Beach,2300.0,NaN,1983.0,NaN,328.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/112...

## Time on Market Calculator

In [80]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 11, 1) ## List (Earlier) date
date2 = datetime(2025, 7, 30) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

271


## Clean CSV for Datawrapper Chart

In [149]:
chart_df = df_top_ten

In [150]:
chart_df = chart_df.fillna(" ")

In [151]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [152]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [153]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [154]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [155]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [156]:
chart_df['PPSF'].astype(float)

0     915.0
1     738.0
2    1033.0
3     625.0
4     675.0
5     411.0
6     396.0
7     328.0
8     579.0
9     363.0
Name: PPSF, dtype: float64

In [157]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [158]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [159]:
csv_date_string = today.strftime("%m_%d_%Y")

In [160]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [161]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-07-30,2770,2770 S Ocean Blvd Unit 201n in Palm Beach,"$4,000,000","4,370",$915,1991,"Carole Koeppel Sotheby's Intl. Realty, Inc.",Dana Koch The Corcoran Group
1,2025-07-30,OCEANS EDGE,2600 S Ocean Blvd Unit 9C & 9D in Boca Raton,"$3,100,000","4,202",$738,1974,Devin Kay Douglas Elliman,
2,2025-07-31,Captains Way AT Admirals Cove Condo,701 Captains Way #701 in Jupiter,"$2,100,000","2,033","$1,033",1988,"Lisa Fishman NV Realty Group, LLC",Jessica Kaufman Beachfront Realty Inc
3,2025-07-31,Peppertree,19630 Sawgrass Cir #2901 in Boca Raton,"$1,300,000","2,080",$625,1979,Judith Romanow Boca West Realty LLC Ina Tuchma...,Ina Tuchman Boca West Realty LLC
4,2025-07-29,Ocean Terrace North Condo,3115 S Ocean Blvd #1004 in Highland Beach,"$1,250,000","1,852",$675,1981,Colleen Newland Douglas Elliman,Scott Correale One Sotheby's International Rea...
5,2025-07-31,Jupiter Country Club Condo II,155 Tresana Blvd #107 in Jupiter,"$1,250,000","3,042",$411,2017,"Gray Welton NV Realty Group, LLC",Jennifer Friedman Fine Living Realty of Palm B...
6,2025-08-01,Braemar Isle Condo,4740 S Ocean Blvd #1403 in Highland Beach,"$812,000","2,050",$396,1982,Ryan Coleman Compass Florida LLC Scott Gerow C...,Mary Ann Phillips RE/MAX Select Group
7,2025-07-30,North Palm Beach,11282 Glen Oaks Ct in North Palm Beach,"$755,000","2,300",$328,1983,LILLIAN PULITZER Brown Harris Stevens of Palm ...,LILLIAN PULITZER Brown Harris Stevens of Palm ...
8,2025-07-31,Two City Plaza Condo,701 S Olive Ave #202 in West Palm Beach,"$700,000","1,208",$579,2008,Rhys Hollyman Logan Realty Inc,Lori Cronican Douglas Elliman Christopher Cron...
9,2025-07-31,Quail Ridge,3579 Quail Ridge Dr Unit Bobwhite A in Boynton...,"$630,000","1,737",$363,1978,Ann Jara The Club at Quail Ridge Realty Inc,Ann Jara The Club at Quail Ridge Realty Inc
